In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

import requests
from urllib.parse import urlsplit
import os
import PyPDF2
import pandas as pd

# to print out all the outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# How many pages are there?
NUM_PAGES = 9                                 # must be there more than 1

In [ ]:
# Open browser
driver = webdriver.Chrome()

links = []

for page_num in range(0,NUM_PAGES+1):
    driver.get("https://www.fdicoig.gov/reports-publications/bank-failures?f%5B0%5D=report_sub_type%3A28&page="+str(page_num))

    # Locate all parent elements (e.g., with class name 'parent-class')
    parent_elements = driver.find_elements(By.CLASS_NAME, 'file--application-pdf')
    
    # Iterate over each parent element
    for parent_element in parent_elements:
        # Find all child elements with the tag 'a' within the parent element
        child_elements = parent_element.find_elements(By.TAG_NAME, 'a')
        
        # Iterate over the child elements and get their href attribute values
        href_values = [element.get_attribute('href') for element in child_elements]
         
        for link in href_values:
            links.append(link)
            print("Appended " + link)


# Open browser
driver = webdriver.Chrome()

In [ ]:
print("Number of files to read and download: " + str(len(links)))

In [ ]:
# Open browser
driver = webdriver.Chrome()

# Iterate over each pdf's link
for link in links:
    driver.get(link)

    # Extract filename from the URL
    path = urlsplit(link).path
    filename = os.path.basename(path)
    
    # Download the PDF
    response = requests.get(link)
    
    # Save the PDF to a file
    with open("../data/" + filename, 'wb') as file:
        file.write(response.content)
    
    print(f'Downloaded file saved as: {filename}')

# CLose browser
driver.quit()

## Read the pdf files

In [ ]:
filenames = []

# Iterate over each pdf's link
for link in links:

    # Extract filename from the URL
    path = urlsplit(link).path
    filename = os.path.basename(path)
    filenames.append(filename)

pdf_texts = {}

for file_name in filenames:
    try:
        with open("../data/" + file_name, 'rb') as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in reader.pages:
                text += page.extract_text()  # Extract text from each page
            pdf_texts[file_name] = text
    except Exception as e:
        pdf_texts[file_name] = f"Error reading file: {e}"

In [ ]:
df = pd.DataFrame(pdf_texts.items(), columns=["filename", "text"])

output_dir = "../data/"

csv_path = os.path.join(output_dir, "output.csv")
df.to_csv(csv_path, index=False, quotechar='"')  # Save with double quotes around entries
print(f"CSV file created at: {csv_path}")

In [ ]:
df.tail(2)